In [14]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

In [15]:
import yaml

In [16]:
from pprint import pprint

In [17]:
import json

In [18]:
from langchain.prompts import HumanMessagePromptTemplate

In [19]:
def get_fore_context(inputs):
    return inputs[: inputs.find("{end token}")].replace("{start token}","")

In [ ]:
inputs = '{start token}import numpy as np\nimport scipy as sp\n{end token}def hello_world():\n    print("Hello world"){middle token}'
data = {"inputs": inputs, "parameters": {"max_new_tokens": 256}}

In [ ]:
get_fore_context(inputs)

'import numpy as np\nimport scipy as sp\n'

In [30]:
class LLM:
    verbose = False
    def __init__(self, conf_file) -> None:
        with open(conf_file, "r", encoding="utf-8") as f:
            conf = yaml.safe_load(f.read())
        OPENAI_API_KEY = conf["OPENAI_API_KEY"]
        OPENAI_API_BASE = conf["OPENAI_API_BASE"]
        OPENAI_API_VERSION = conf["OPENAI_API_VERSION"]
        DEPLOYMENT = conf["DEPLOYMENT"]
        self.llm = AzureChatOpenAI(
            openai_api_type="azure",
            openai_api_version=OPENAI_API_VERSION,
            openai_api_base=OPENAI_API_BASE,
            openai_api_key=OPENAI_API_KEY,
            deployment_name=DEPLOYMENT,
            temperature=0,
        )

    def complete(self, query):
        """Answer to a query."""
        system_setting = SystemMessage(
            content="You help to complete the user's code.")
        
        message = HumanMessage(content=query)
        return self.llm.invoke([system_setting, message]).content
    
    def complete_code(self, code_context):
        """Take the input from the request and output.

        args:
            code_context(str): the code_context

        return(dict): the response
        """
        fore_context = get_fore_context(code_context)
        system_setting = SystemMessage(
            content="You are a code autocompleter.")
        prompt = f"""
        Please complete code for the following code. Make code completion after the end token.
        \n\n
        {fore_context}
        """
        message = HumanMessage(content=prompt)
        if LLM.verbose:
            from pprint import pprint
            pprint(prompt)
        return self.llm.invoke([system_setting, message]).content

llm = LLM(".env-35-16k.yml")

In [32]:
LLM.verbose = True
pprint(llm.complete_code(inputs))

('\n'
 '        Please complete code for the following code. Make code completion '
 'after the end token.\n'
 '        \n'
 '\n'
 '\n'
 '        import numpy as np\n'
 'import scipy as sp\n'
 '\n'
 '        ')
('import matplotlib.pyplot as plt\n'
 '\n'
 '# Generate some random data\n'
 'x = np.linspace(0, 10, 100)\n'
 'y = np.sin(x)\n'
 '\n'
 '# Plot the data\n'
 'plt.plot(x, y)\n'
 "plt.xlabel('x')\n"
 "plt.ylabel('y')\n"
 "plt.title('Plot of y = sin(x)')\n"
 'plt.show()')
